In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date, timedelta

repo_root = Path().resolve().parent
sys.path.append(str(repo_root))
import src.fetch_info as fetch

In [ ]:
table = pd.read_html('table.html', encoding='utf-8')[0]
all = table['Mã CK']

#VN30 = list.symbols_by_group('VN30').tolist()
#all = [t for t in VN30]

In [ ]:
from vnstock import Listing
list = Listing(source='vci')


In [ ]:
#tickers = [t for t in VN30]
tickers = [t for t in all]

day_range = 360


In [ ]:
from time import sleep
panel = {}

stock = fetch.fetch_stock(tickers, hist=day_range, combine = False, interval='1d')




In [ ]:
import pickle

with open('HSC_ML.pkl', 'wb') as file:
    pickle.dump(stock, file)

In [39]:
with open('HSC_ML.pkl', 'rb') as file:
    panel = pickle.load(file)

panel

for t in panel:
    stock = panel[t]
    stock['ret'] = stock['close'].pct_change()
    #stock['abs_ret'] = abs(stock['ret'])
    stock['ret_lag1'] = stock['ret'].shift(1)
    stock['ret_lag2'] = stock['ret'].shift(2)

print(panel)

{np.str_('ACB'):           time   open   high    low  close   volume       ret  ret_lag1  \
0   2024-12-19  20.93  20.93  20.64  20.72  9720556       NaN       NaN   
1   2024-12-20  20.76  20.80  20.68  20.72  4729595  0.000000       NaN   
2   2024-12-23  20.76  20.85  20.72  20.80  3469073  0.003861  0.000000   
3   2024-12-24  20.80  20.89  20.72  20.89  3233697  0.004327  0.003861   
4   2024-12-25  20.89  21.39  20.85  21.26  9518178  0.017712  0.004327   
..         ...    ...    ...    ...    ...      ...       ...       ...   
254 2025-12-26  23.90  24.05  23.65  23.90  7831900 -0.004167 -0.012346   
255 2025-12-29  23.85  24.10  23.80  24.00  3770400  0.004184 -0.004167   
256 2025-12-30  24.00  24.20  23.90  24.10  5337300  0.004167  0.004184   
257 2025-12-31  24.15  24.25  23.90  24.00  5932900 -0.004149  0.004167   
258 2026-01-05  24.00  24.10  23.95  24.00  3388000  0.000000 -0.004149   

     ret_lag2  
0         NaN  
1         NaN  
2         NaN  
3    0.000000  
4 

In [40]:
# Condition 1: Change > 6%, 3 consecutive days 
crit = 6
parent = []
for t in panel:
    crit_pct = crit / 100
    stock = panel[t]
    stock['ticker'] = t

    stock = stock[(stock['ret'] >= crit_pct) &
                   (stock['ret_lag1'] >= crit_pct) & 
                   (stock['ret_lag2'] >= crit_pct)]
    
    parent.append(stock)

    final = pd.concat(parent, ignore_index = True)


In [42]:
crit = 18  # use 0.15 if returns are in decimals
parent = []

crit_pct = crit/100

for name, stock in panel.items():  # panel must be a dict/Series of DataFrames
    cols = ["ret", "ret_lag1", "ret_lag2"]
    missing = set(cols) - set(stock.columns)
    if missing:
        raise KeyError(f"Missing columns in {name}: {missing}")

    mask = stock[cols].sum(axis=1) > crit_pct
    if mask.any():
        parent.append(stock.loc[mask].assign(ticker=name))

final2 = pd.concat(parent, ignore_index=True) if parent else pd.DataFrame()

In [ ]:
final.sort_values(['time'], ascending=False)
#final.to_excel("criteria1.xlsx")
final

In [43]:
final2.sort_values(['time'], ascending=False)
#final2.to_excel("criteria2.xlsx")
final2

,time,open,high,low,close,volume,ret,ret_lag1,ret_lag2,ticker
0,2025-04-14,14.04,14.62,13.84,14.62,1559973,0.067153,0.067810,0.069167,ANV
1,2025-07-01,21.30,21.88,20.43,21.88,8837766,0.068359,0.068336,0.050411,ANV
2,2025-07-02,22.17,23.38,21.88,22.85,8080154,0.044333,0.068359,0.068336,ANV
3,2025-04-14,31.05,32.03,30.56,31.83,905128,0.031432,0.074887,0.097058,BVS
4,2025-04-14,11.81,12.00,10.86,12.00,6233260,0.016088,0.097584,0.096840,CEO
5,2025-06-02,15.91,17.24,15.52,17.05,27500032,0.085296,0.057201,0.053901,CEO
6,2025-07-18,20.00,22.00,20.00,21.52,33177665,0.076000,0.099505,0.010556,CEO
7,2025-10-15,30.40,31.80,29.70,31.70,22329700,0.042763,0.048276,0.098485,CEO
8,2025-08-11,18.70,18.74,18.40,18.74,32425592,0.069635,0.068944,0.066363,CII
9,2025-08-12,19.38,20.02,18.25,20.02,69094709,0.068303,0.069635,0.068944,CII


In [ ]:
# HSX: 10+10+4 (dung ti le: 7/7/2)
# Check marketcap

# Hypothesis: san HNX/HSX
# Theo nganh
#